<a href="https://colab.research.google.com/github/Ali-Kazmi/All-Pairs-Shortest-Path/blob/master/APSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will use python's networkX library to perform All Pairs Shortest Paths (APSP) on the california road dataset. Then, i'll use scipy to do the same much faster. Lastly, I'll implement my own and use numba to speed it up, beating scipy! 

In [2]:
# Get the data 
from pandas import read_csv
OLcnode = read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cnode',
                 names=['Node_ID', 'X','Y'],
                 delim_whitespace=True, nrows=5000) 

OLcedge=read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cedge',
                 names=['Edge ID', 'Start_Node_ID','End_Node_ID','L2_distance'],
                 delim_whitespace=True,nrows=5000) 


In [ ]:
OLcedge.head() 

,Edge ID,Start_Node_ID,End_Node_ID,L2_distance
0,0,1609,1622,57.403187
1,1,2471,2479,29.718756
2,2,2463,2471,61.706902
3,3,2443,2448,19.080025
4,4,1417,1491,28.248583


In [ ]:
OLcnode.head()

,Node_ID,X,Y
0,0,769.948669,2982.984131
1,1,863.275757,3005.275635
2,2,690.196411,3333.704834
3,3,1197.556519,2984.470215
4,4,1261.188599,2985.956299


The cell below will output an example of one shortest path (between two vertices). For APSP, we will be finding these shortest paths for all shortest paths between all pairs of vertices. As you can imagine, it gets expensive fast 

In [4]:
def get_edgelist(df):
    ### BEGIN SOLUTION
    return [(a, b, {'w': w}) for a, b, w in zip(df["Start_Node_ID"], df["End_Node_ID"], df["L2_distance"])]
    ### END SOLUTION
    
# Demo
edgelist = get_edgelist(OLcedge)
edgelist[:5] #just display a few. There's a lot of these! 

[(1609, 1622, {'w': 57.403187}),
 (2471, 2479, {'w': 29.718756}),
 (2463, 2471, {'w': 61.706902}),
 (2443, 2448, {'w': 19.080025}),
 (1417, 1491, {'w': 28.248583})]

In [5]:

from networkx import Graph
G = Graph()
G.add_nodes_from(OLcnode["Node_ID"])
G.add_edges_from(edgelist)

print(f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

The graph has 5590 nodes and 4995 edges.


In [6]:
import numpy as np
from networkx import to_numpy_array
adjlist = to_numpy_array(G)

In [ ]:
pathfwnp # This is just to give you an idea of the output, and shows what a proper result will be 

matrix([[ 0.,  1.,  1., ..., inf, inf, inf],
        [ 1.,  0.,  2., ..., inf, inf, inf],
        [ 1.,  2.,  0., ..., inf, inf, inf],
        ...,
        [inf, inf, inf, ...,  0.,  2.,  3.],
        [inf, inf, inf, ...,  2.,  0.,  1.],
        [inf, inf, inf, ...,  3.,  1.,  0.]])

time: 3.93 ms


In [ ]:
import scipy as sp
import scipy.sparse 

%timeit sp.sparse.csgraph.shortest_path(adjlist,method='FW')

1 loop, best of 3: 376 ms per loop



Looking into how Scipy does it https://github.com/scipy/scipy/blob/master/scipy/sparse/csgraph/_shortest_path.pyx, the trick is using cython (it compiles to C) and numpy. They also use custom datatypes. Below this, i'm going to make improvements to the scipy (parallelize it, and then make it use min + semiring). Together these can get around a 30% speedup. See the experiments page of the github wiki for the results  

In [7]:
%load_ext Cython

In [160]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
%%cython
import warnings

import numpy as np
cimport numpy as np

from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999

class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    cdef unsigned int i, j, k

    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk

In [161]:
%timeit floyd_warshall_scipy(adjlist)

1 loop, best of 3: 1min 43s per loop


In [8]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
#%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
%%cython -f
import warnings
from cython.parallel import prange
import cython.parallel as cp
import numpy as np
cimport numpy as np
cimport openmp
from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999


class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy_parallel(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    #cdef unsigned int i, j, k
    cdef int i, j, k
    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in prange(N,nogil=True):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in prange(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk

In [9]:
%timeit floyd_warshall_scipy_parallel(adjlist)

1 loop, best of 3: 1min 5s per loop


In [158]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
#%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
%%cython -f
import warnings
from cython.parallel import prange
import cython.parallel as cp
import numpy as np
cimport numpy as np
cimport openmp
from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999


class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy_parallel_semir(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    #cdef unsigned int i, j, k
    cdef int i, j, k
    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in prange(N,nogil=True):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in prange(N):
                    dist_matrix[i,j]=min(dist_matrix[i,j],dist_matrix[i, k] + dist_matrix[k, j])

In [159]:
%timeit floyd_warshall_scipy_parallel_semir(adjlist)

1 loop, best of 3: 1min 40s per loop


In [95]:
print(floyd_warshall_scipy_parallel_semir(adjlist))

[[ 0.  1.  1. ... inf inf  5.]
 [ 1.  0.  2. ... inf inf  6.]
 [ 1.  2.  0. ... inf inf  4.]
 ...
 [inf inf inf ...  0.  1. inf]
 [inf inf inf ...  1.  0. inf]
 [ 5.  6.  4. ... inf inf  0.]]


In [ ]:
#correctness checks 
output1=floyd_warshall_scipy(adjlist)
output2=floyd_warshall_scipy_parallel(adjlist)
output3=floyd_warshall_scipy_parallel_semir(adjlist)

In [ ]:
#yay we passed
print((output1==output2).all())
print((output3==output2).all())

True
True


Now, i'm going to be seeing how fast numpy performs with similar code. 

In [152]:
import numpy as np 
from numba import jit
#assume its not sparse, no negative cycles, etc 
@jit(nopython=True,parallel=False)
def numbafw(csgraph):
  dist_matrix=csgraph
  #dist_matrix[dist_matrix==0] = np.inf have to do this by hand 
  for a,v in np.ndenumerate(dist_matrix):
    if v ==0:
      dist_matrix[a]=np.inf
  _fw(dist_matrix)  
  return dist_matrix

@jit(nopython=True,parallel=False)
def _fw(dist_matrix):
  N=dist_matrix.shape[0]
  assert dist_matrix.shape[1]==N
  #  Initialize distance matrix
  #   - set diagonal to zero
  #   - symmetrize matrix if non-directed graph is desired
  #dist_matrix.flat[::N + 1] = 0
  np.fill_diagonal(dist_matrix,0) #does this work?
  for i in range(N):
      for j in range(i + 1, N):
          if dist_matrix[j, i] <= dist_matrix[i, j]:
              dist_matrix[i, j] = dist_matrix[j, i]
          else:
              dist_matrix[j, i] = dist_matrix[i, j]
  #below is the FW
  for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == np.inf:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
    

In [153]:
%timeit answerfornumbafw=numbafw(adjlist)

1 loop, best of 3: 1min 40s per loop


In [91]:
print(adjlist)
print(numbafw(adjlist)) # this is a small correctness example. It matches what scipy produces for the same input 

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[ 0.  1.  1. ... inf inf  5.]
 [ 1.  0.  2. ... inf inf  6.]
 [ 1.  2.  0. ... inf inf  4.]
 ...
 [inf inf inf ...  0.  1. inf]
 [inf inf inf ...  1.  0. inf]
 [ 5.  6.  4. ... inf inf  0.]]


In [145]:
import numpy as np 
import numba
from numba import jit,prange
#assume its not sparse, no negative cycles, etc 
@jit(nopython=True)
def numbafwparallel(csgraph):
  dist_matrix=csgraph
  #dist_matrix[dist_matrix==0] = np.inf have to do this by hand 
  for a,v in np.ndenumerate(dist_matrix):
    if v ==0:
      dist_matrix[a]=np.inf
  _fwp(dist_matrix)  #dist_matrix[dist_matrix == 0] = np.inf
  return dist_matrix

@jit(nopython=True,parallel=True)
def _fwp(dist_matrix):
  N=dist_matrix.shape[0]
  assert dist_matrix.shape[1]==N
  #  Initialize distance matrix
  #   - set diagonal to zero
  #   - symmetrize matrix if non-directed graph is desired
  #dist_matrix.flat[::N + 1] = 0
  np.fill_diagonal(dist_matrix,0) #does this work?
  for i in range(N):
      for j in range(i + 1, N):
          if dist_matrix[j, i] <= dist_matrix[i, j]:
              dist_matrix[i, j] = dist_matrix[j, i]
          else:
              dist_matrix[j, i] = dist_matrix[i, j]
  #below is the FW
  for k in range(N):
    for i in numba.prange(N):
        if dist_matrix[i, k] == np.inf:
            continue
        for j in numba.prange(N):
            d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
            if d_ijk < dist_matrix[i, j]:
                dist_matrix[i, j] = d_ijk
    

In [146]:
%timeit answerfornumbafwparallel=numbafwparallel(adjlist)

1 loop, best of 3: 1min 36s per loop


In [129]:
import numpy as np 
from numba import jit,prange
#assume its not sparse, no negative cycles, etc 
@jit(nopython=True)
def numbafwparallelsemir(csgraph):
  dist_matrix=csgraph
  #dist_matrix[dist_matrix==0] = np.inf have to do this by hand 
  for a,v in np.ndenumerate(dist_matrix):
    if v ==0:
      dist_matrix[a]=np.inf
  _fwps(dist_matrix)  #dist_matrix[dist_matrix == 0] = np.inf
  return dist_matrix

@jit(nopython=True,parallel=True)
def _fwps(dist_matrix):
  N=dist_matrix.shape[0]
  assert dist_matrix.shape[1]==N
  #  Initialize distance matrix
  #   - set diagonal to zero
  #   - symmetrize matrix if non-directed graph is desired
  #dist_matrix.flat[::N + 1] = 0
  np.fill_diagonal(dist_matrix,0) #does this work?
  for i in range(N):
      for j in range(i + 1, N):
          if dist_matrix[j, i] <= dist_matrix[i, j]:
              dist_matrix[i, j] = dist_matrix[j, i]
          else:
              dist_matrix[j, i] = dist_matrix[i, j]
  #below is the FW
  for k in range(N):
            for i in prange(N):
                if dist_matrix[i, k] == np.inf:
                    continue
                for j in prange(N):
                    dist_matrix[i,j]=min(dist_matrix[i,j],dist_matrix[i, k] + dist_matrix[k, j])
    

In [141]:
%timeit answerfornumbafwparallelsemir=numbafwparallelsemir(adjlist)

1 loop, best of 3: 1min 35s per loop


In [142]:
cythonans=floyd_warshall_scipy_parallel_semir(adjlist)
answerfornumbafwparallelsemir=numbafwparallelsemir(adjlist)
print(cythonans==answerfornumbafwparallelsemir)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [56]:
!pip install numba